## 웹 크롤링
- 웹에서 데이터 수집

## 정적/동적인 웹페이지
- 페이지를 궛ㅇ하는 요소드을 한번에 웹 서버로부터 가져옴
- 사용자의 반응에 따라 실시간으로 데이터를 가져옴

## URL 분석
- 페이지를 주소에서 '?' 전 까지는 서버주소
- '?' 아래는 모두 패러미터 (필수 패러미터 찾기)
- 최대한 수집할 데이터가 존재하는 페이지의 url을 자유롭게 만들 수 있도록 준비

네이버 날씨 url 

https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%82%A0%EC%94%A8&ackey=o1b0nnoi

search.naver.com -> 메인 페이지 (검색 페이지)

/ -> 페이지 표시

? 앞 -> 서버 주소 -> 수정 x

? 뒤 -> 옵션

**query -> 검색어 -> query가 가장 중요 -> query 뒷 부분이 변수가 된다**

%EB%82%A0%EC%94%A8 -> 날씨 -> 유니코드로 작성


**query 만 제외하고 다 지우기**

변경한 url = https://search.naver.com/search.naver?query=%EB%82%A0%EC%94%A8

In [1]:
cat_id = {'스킨케어':'104001', '마스크팩':'104002'}
url = "https://www.musinsa.com/main/beauty/ranking?"
print(url+cat_id['스킨케어'])

https://www.musinsa.com/main/beauty/ranking?104001


# 네이버 날씨 추출 (static)
- https://search.naver.com/search.naver?query=%EB%82%A0%EC%94%A8
- requests 패키지로 페이지의 모든 html을 호출한다

In [ ]:
import requests

url = "https://search.naver.com/search.naver?query=%EB%82%A0%EC%94%A8"

# 위 사이트의 html 호출
html = requests.get(url)

# 호출결과 확인
html

# 호출결과 -> Response [200] (정상)

<Response [200]>

In [ ]:
# html이 출력되는지만 확인 
# 하단에 text editor 누르면 hmtl 코드 전체 확인 가능
print(html.text)

<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="strict-origin-when-cross-origin">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta property="og:title" content="날씨 : 네이버 검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="'날씨'의 네이버 검색 결과입니다."> <meta name="description" lang="ko" content="'날씨'의 네이버 검색 결과입니다."> <title>날씨 : 네이버 검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_32x32_240820.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_251218.css"> <link rel="stylesheet" type="te

In [11]:
# 파싱 html -> python
from bs4 import BeautifulSoup

soup = BeautifulSoup(html.text, 'html.parser')
soup

<!DOCTYPE html>
 <html lang="ko"><head> <meta charset="utf-8"/> <meta content="strict-origin-when-cross-origin" name="referrer"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <meta content="날씨 : 네이버 검색" property="og:title"> <meta content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png" property="og:image"/> <meta content="'날씨'의 네이버 검색 결과입니다." property="og:description"/> <meta content="'날씨'의 네이버 검색 결과입니다." lang="ko" name="description"/> <title>날씨 : 네이버 검색</title> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_32x32_240820.ico" rel="shortcut icon"/> <link href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" rel="search" title="Naver" type="application/opensearchdescription+xml"><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_251218.css" rel="stylesheet" type="text/css"/> <link href="https://ssl.pst

In [13]:
# 원하는 데이터(온도) 찾기
온도html = soup.find('div', 'temperature_text')
온도html.text     

# <div class="temperature_text">22.0°</div>
# 여기서 온도는 <> 태그 밖에 있으므로 text로 추출

' 현재 온도-4.8° '

In [29]:
# 문자열 조작
온도 = 온도html.text
print(온도)   # ' 현재온도-4.8° '
print("=" * 20)

# 온도[6:]는 위험함 -> 문자열 슬라이싱이기 때문에 여름되면 3자리 나올수도 있음
# ' 현재 온도-4.8° ' -> 앞 뒤 공백 제거

온도.strip().replace('현재 온도', '')[:-1]    #'4.8°'


 현재 온도-3.7° 


'-3.7'

In [2]:
# 웹 크롤릴 용 전체 코드 
import requests
from bs4 import BeautifulSoup

url = "https://search.naver.com/search.naver?query=%EB%82%A0%EC%94%A8"

html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

# 온도
온도html = soup.find('div', 'temperature_text')

# 날씨 상태
날씨상태 = soup.find('span', 'weather before_slash').text

# 습도 (find all 사용)
습도 = soup.find_all('dd', 'desc')

온도 = 온도html.text

온도.strip().replace('현재 온도', '')[:-1]

print(f"{온도}이고, 날씨 상태는 {날씨상태} 입니다.")
print(습도[1].text)

 현재 온도-3.0° 이고, 날씨 상태는 맑음 입니다.
23%


# 배열 형태의 데이터 찾기 - find all
- 같은 태그명과 속성을 가진 데이터가 여러 개 -> 마치 배열 같은 구조

In [3]:
날씨리스트 = soup.find_all('dd', 'desc')
for i in 날씨리스트:
    print(i)

<dd class="desc">-6.1°</dd>
<dd class="desc">23%</dd>
<dd class="desc">2.1m/s</dd>
<dd class="desc">40%</dd>
<dd class="desc">40%</dd>
<dd class="desc">60%</dd>
<dd class="desc">60%</dd>


In [68]:
# 클래스 이름 summary_list 인 경우
len(soup.find_all('dl', 'summary_list'))        # 5개

5

In [85]:
soup.find('dl', class_='summary_list') 

<dl class="summary_list"> <div class="sort"> <dt class="term">체감</dt> <dd class="desc">-6.1°</dd> </div> <div class="sort"> <dt class="term">습도</dt> <dd class="desc">23%</dd> </div> <div class="sort"> <dt class="term">북서풍</dt> <dd class="desc">2.1m/s</dd> </div> </dl>

[chatgpt 질문]
- 코드 추출 요청
- soup.find('dl', class_='summary_list') 코드 실행 시 다음 html이 출력됨

- html: <dl class="summary_list"> <div class="sort"> <dt class="term">체감</dt> <dd class="desc">-6.1°</dd> </div> <div class="sort"> <dt class="term">습도</dt> <dd class="desc">23%</dd> </div> <div class="sort"> <dt class="term">북서풍</dt> <dd class="desc">2.1m/s</dd> </div> </dl>

- 딕셔너리 = {체감: -6.8, 습도:25%} 와 같은 형태로 만들고 싶음.

In [8]:
# 작성된 코드
dl = soup.find('dl', class_='summary_list')

result = {}

for item in dl.find_all('div', class_='sort'):
    key = item.find('dt', class_='term').get_text(strip=True)
    value_text = item.find('dd', class_='desc').get_text(strip=True)
    result[key] = value_text
result

{'체감': '-6.1°', '습도': '23%', '북서풍': '2.1m/s'}

In [9]:
# 데이터프레임으로 변환_가로형 (위 코드 실행 후 실행)
import pandas as pd
df = pd.DataFrame([result])
df

,체감,습도,북서풍
0,-6.1°,23%,2.1m/s


In [10]:
# 데이터프레임으로 변환_세로형
import pandas as pd
df = pd.DataFrame(result.items(), columns=['구분', '값'])
df

,구분,값
0,체감,-6.1°
1,습도,23%
2,북서풍,2.1m/s
